In [1]:
from waylay import WaylayClient, RestResponseError
waylay_client = WaylayClient.from_profile('io')

# Waylay Python SDK - Making REST calls

The Waylay Python SDK exposes a number of _REST Services_, each a collection of _REST Resources_ that have _action methods_.

For an overview, see 
> `[IO]` https://docs-io.waylay.io/#/api/sdk/python<br>
> `[Enterprise]` https://docs.waylay.io/api/sdk/python/

In most cases these _action methods_ directly correspond to an underlying REST endpoint of the waylay platform.

In general, the Python SDK takes care of
* authentication
* the endpoint url to call, and HTTP method to use
* converting JSON responses to Python objects, extracting the relevant data
* converting Python request body objects to JSON
* handling errors 

Check out the documentation sites ([io](https://docs-io.waylay.io/#/api/sdk/python), [enterprise](https://docs.waylay.io/api/sdk/python/)) on

* how requests and response are exactly represented as json
* which additional parameters can be provided
* what the preconditions and effects are of your call

### request arguments
These are the general rules for making requests, corresponding to the REST documentation of the underlying REST action:
* url **path parameters** are passed as **positional arguments**:
  ###### example

  > `waylay_client.queries.query.get('151CF-temperature')` 

  will use 151CF-temperature to construct a request to fetch this query definition

  > `GET .../config/query/151CF-temperature` 

* **query parameters** belong in a named **`params` argument**:
  ###### example
  > `waylay_client.queries.query.execute('151CF-temperature', params={'from':'2021-03-01T10:00:00+00:00'})`
  
  will bind the `from` parameter

* **request objects** are passed into a **`body` argument**:
  ###### example
  > `waylay_client.queries.query.execute(body={'resource':'RDJ_89839','metric':'revolutions'})`}|

### response handling
These are the general rules on how the SDK treats results of REST calls:
* Only successfull responses (HTTP status `2XX`) return a result, other responses raise an exception.
* The SDK extracts the most relevant part of the (json) response body. You can use the `select_path` argument to override this behaviour.
* When the REST call produces **timeseries data**, the SDK will return a **pandas DataFrame**.
* By providing the  `raw=True` argument, you instruct the SDK to skip all error and response handling, and **just return the HTTP response**.

### error handling

The SDK uses the following exception hierarchy to notify problems. These exception classes belong to the `waylay.exception` module. They all descend from a `WaylayError` base class.

| exception class | raised in case of |
| --------------- | ----------- |
| `AuthError` | Waylay authentication errors. |
| `ConfigError` |  Waylay client configuration errors. |
| `RequestError` | Errors in tools and utilities that are not directly related to a REST call. |
| `RestRequestError` | Failure to prepare a REST call. |
| `RestResponseError` | Wraps the result of a failed REST call. |
| `RestResponseParseError` | Failure to parse the result of a succesfull REST call. |

Errors of type `RestResponse` have the following attributes that you can use to handle problems:

* `response` contains the full HTTP Response object of the REST call, which lets you inspect the status code (`response.status_code`) , response body (`response.body`) and other attributes such as _headers_.
* `message` will give you the most relevant error message




### http response information

Most _action methods_ support a `raw=True` parameter. This will prevent exception handling and parsing of the REST call by the SDK. The unparsed result and http response information is returned in a _Response_ object with attributes
 * `body` : the result data (json and csv data is parsed to python data structures)
 * standard http information such as `url`, `method`, `headers`, `status_code`, `client_response`

In [2]:
waylay_client.queries.query.list()

['hvac_0001',
 'plant-monitor-overview',
 'query-byoml-tutorial-2021-06-23-0201',
 'query-byoml-tutorial-2021-06-23-0698',
 'weather_query']

In [3]:
http_response = waylay_client.queries.query.get('plant-monitor-overview', raw=True)
{
    'url':http_response.url,
    'method':http_response.method,
    'content-type':http_response.headers['content-type'],
    'status_code':http_response.status_code,
    'name':http_response.body['name'],
}


{'url': 'https://api-io.waylay.io/queries/v1/query/plant-monitor-overview',
 'method': 'GET',
 'content-type': 'application/json',
 'status_code': 200,
 'name': 'plant-monitor-overview'}

### access http response information from an error
When an request is unsuccessfull, the client will raise an exception. 

These exceptions (from the `waylay.exceptions` module) are either (instances of subclasses of)
* a `RestRequestError` that indicates a problem before sending an api call to waylay (e.g. when input argument conversion fails)
* a `RestResponseError` that reports a problem from or after the api call to waylay. This exception gives you access to the underlying response (`response` attribute)
  * a `RestResponseParseError` error indicates a problem in processing a succesfull response from the waylay platform. All other `RestResponseError` will come from errors reported by the waylay platform itself (http status code above the `200` range).
  
Other errors can occur (such as standard python `ValueError`,`TypeError` or `AttributeError`) but these will normally indicate a programming error. Networking failures will normally result in a `ClientConnectionError`

In [4]:
# try to get the representation of a `query` entity that does not exist. 
# this will result in a `404 NOT FOUND` error 
try:
   waylay_client.queries.query.get('where are you???')
except RestResponseError as exc:
   print(exc.message)
   print(exc.response.status_code)
   print(exc.response.url)
    


operation=not_found_error
404
https://api-io.waylay.io/queries/v1/query/where%20are%20you


## accessing the complete response
The SDK extracts the most relevant part of the (json) response body. You can use the `select_path` argument to override this behaviour

In [5]:
# the `query` entity in the REST response is given as default response on the SDK action
waylay_client.queries.query.get('plant-monitor-overview')

{'until': 'now',
 'data': [{'metric': 'moisture',
   'resource': '0086b26b-5ae7-439b-b3da-35c0dd227a0f',
   'aggregation': 'mean'},
  {'metric': 'temperature',
   'resource': '0086b26b-5ae7-439b-b3da-35c0dd227a0f',
   'aggregation': 'mean'},
  {'metric': 'humidity',
   'resource': '0086b26b-5ae7-439b-b3da-35c0dd227a0f',
   'aggregation': 'mean'},
  {'metric': 'light',
   'resource': '0086b26b-5ae7-439b-b3da-35c0dd227a0f',
   'aggregation': 'mean'}],
 'window': 'PT24H'}

In [6]:
waylay_client.queries.query.get('plant-monitor-overview', select_path=False)

{'_links': {'self': {'href': 'https://api-io.waylay.io/queries/v1/query/plant-monitor-overview'},
  'execute': {'href': 'https://api-io.waylay.io/queries/v1/data/plant-monitor-overview'}},
 'attrs': {'created': '2021-05-09T16:14:47.631611+00:00',
  'created_by': 'users/89b417eb-5a88-4410-9adb-e3a9c393cc8f',
  'modified': '2021-05-09T16:14:47.631611+00:00',
  'modified_by': 'users/89b417eb-5a88-4410-9adb-e3a9c393cc8f',
  'path': '/tsa/query/plant-monitor-overview'},
 'name': 'plant-monitor-overview',
 'meta': {},
 'query': {'until': 'now',
  'data': [{'metric': 'moisture',
    'resource': '0086b26b-5ae7-439b-b3da-35c0dd227a0f',
    'aggregation': 'mean'},
   {'metric': 'temperature',
    'resource': '0086b26b-5ae7-439b-b3da-35c0dd227a0f',
    'aggregation': 'mean'},
   {'metric': 'humidity',
    'resource': '0086b26b-5ae7-439b-b3da-35c0dd227a0f',
    'aggregation': 'mean'},
   {'metric': 'light',
    'resource': '0086b26b-5ae7-439b-b3da-35c0dd227a0f',
    'aggregation': 'mean'}],
  'win

In [7]:
waylay_client.queries.query.get('plant-monitor-overview', select_path=['attrs','modified'])

'2021-05-09T16:14:47.631611+00:00'

## customise or replace dataframe conversions
The methods of the `queries.query` that return timeseries data, will normally return their data as a pandas `DataFrame`. These have immediate display support in jupyter notebooks:

In [8]:
waylay_client.queries.query.execute(
    'plant-monitor-overview', 
    params={ "until": "16-08-2021" }
)

resource                         0086b26b-5ae7-439b-b3da-35c0dd227a0f  \
metric                                                       moisture   
timestamp                                                               
2021-08-15 00:00:03.566000+00:00                                    1   
2021-08-15 00:10:06.316000+00:00                                    0   
2021-08-15 00:20:08.804000+00:00                                    0   
2021-08-15 00:30:12.968000+00:00                                    0   
2021-08-15 00:40:16.387000+00:00                                    0   
...                                                               ...   
2021-08-15 23:19:41.255000+00:00                                    0   
2021-08-15 23:29:43.719000+00:00                                    1   
2021-08-15 23:39:46.340000+00:00                                    0   
2021-08-15 23:49:50.271000+00:00                                    0   
2021-08-15 23:59:53.009000+00:00                                    8   

resource                                                     
metric                           temperature humidity light  
timestamp                                                    
2021-08-15 00:00:03.566000+00:00          25       56     0  
2021-08-15 00:10:06.316000+00:00          25       56     0  
2021-08-15 00:20:08.804000+00:00          25       56     0  
2021-08-15 00:30:12.968000+00:00          24       56     0  
2021-08-15 00:40:16.387000+00:00          24       56     0  
...                                      ...      ...   ...  
2021-08-15 23:19:41.255000+00:00          24       48     0  
2021-08-15 23:29:43.719000+00:00          24       57     0  
2021-08-15 23:39:46.340000+00:00          24       48     0  
2021-08-15 23:49:50.271000+00:00          24       49     0  
2021-08-15 23:59:53.009000+00:00          24       48     0  

[142 rows x 4 columns]

You can use the `response_constructor` parameter to replace the dataframe constructor with your own method. 

Other _action methods_ that apply conversions to raw json data structures normally support this same optional parameter.

A falsy value will let the method return just the json data payload (as python object).

In [9]:
waylay_client.queries.query.execute(
    'plant-monitor-overview', 
    params={ 'until' : "16-08-2021", 'window': 'PT1H'},
    response_constructor=False
)

[{'columns': ['timestamp',
   {'resource': '0086b26b-5ae7-439b-b3da-35c0dd227a0f', 'metric': 'moisture'},
   {'resource': '0086b26b-5ae7-439b-b3da-35c0dd227a0f',
    'metric': 'temperature'},
   {'resource': '0086b26b-5ae7-439b-b3da-35c0dd227a0f', 'metric': 'humidity'},
   {'resource': '0086b26b-5ae7-439b-b3da-35c0dd227a0f', 'metric': 'light'}],
  'data': [[1629068977415, 0, 24, 47, 0],
   [1629069581255, 0, 24, 48, 0],
   [1629070183719, 1, 24, 57, 0],
   [1629070786340, 0, 24, 48, 0],
   [1629071390271, 0, 24, 49, 0],
   [1629071993009, 8, 24, 48, 0]],
  'data_axis': 'column',
  'attributes': {'role': 'input'},
  'window_spec': {'from': 1629068977415,
   'until': 1629072596009,
   'window': 'PT3618.594S',
   'freq': 'PT603S'}}]

In [10]:
# return a map with timestamps as keys, the observation values as value. 
waylay_client.queries.query.execute(
    'plant-monitor-overview', 
    params={ 'until' : "16-08-2021", 'window': 'PT1H'},
    response_constructor=lambda d: { row[0] : row[1:] for row in d[0]['data'] }
)

{1629068977415: [0, 24, 47, 0],
 1629069581255: [0, 24, 48, 0],
 1629070183719: [1, 24, 57, 0],
 1629070786340: [0, 24, 48, 0],
 1629071390271: [0, 24, 49, 0],
 1629071993009: [8, 24, 48, 0]}

In [11]:
import numpy as np
# return timeseries data as a numpy array, transposed as an array per series 
waylay_client.queries.query.execute(
   'plant-monitor-overview', 
    params={ 'until' : "16-08-2021", 'window': 'PT1H'},
    response_constructor=lambda d: np.transpose(d[0]['data'])
)

array([[1629068977415, 1629069581255, 1629070183719, 1629070786340,
        1629071390271, 1629071993009],
       [            0,             0,             1,             0,
                    0,             8],
       [           24,            24,            24,            24,
                   24,            24],
       [           47,            48,            57,            48,
                   49,            48],
       [            0,             0,             0,             0,
                    0,             0]])